[View in Colaboratory](https://colab.research.google.com/github/tcglarry/aoi/blob/master/aoi_densenet_conv_dropout.ipynb)

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

aoi_aoi_test1.ipynb  Sample upload.txt			      Untitled1.ipynb
aoi_test	     TAHU PowerPoint Template.pdf	      Untitled2.ipynb
Capstone_PixNet.zip  TAHU PowerPoint Template - 柱狀圖 1.ods  Untitled3.ipynb
Colab Notebooks      test for excel
PiXNet		     Untitled0.ipynb


In [0]:
!cp drive/aoi_test/train_images.zip .

In [0]:
!cp drive/aoi_test/train.csv .

In [11]:
!unzip train_images.zip


Archive:  train_images.zip
   creating: train_images/
 extracting: train_images/train_00000.png  
 extracting: train_images/train_00001.png  
 extracting: train_images/train_00002.png  
 extracting: train_images/train_00003.png  
 extracting: train_images/train_00004.png  
 extracting: train_images/train_00005.png  
 extracting: train_images/train_00006.png  
 extracting: train_images/train_00007.png  
 extracting: train_images/train_00008.png  
 extracting: train_images/train_00009.png  
 extracting: train_images/train_00010.png  
 extracting: train_images/train_00011.png  
 extracting: train_images/train_00012.png  
 extracting: train_images/train_00013.png  
 extracting: train_images/train_00014.png  
 extracting: train_images/train_00015.png  
 extracting: train_images/train_00016.png  
 extracting: train_images/train_00017.png  
 extracting: train_images/train_00018.png  
 extracting: train_images/train_00019.png  
 extracting: train_images/train_00020.png  
 extracting: train_ima

In [13]:
!ls train_images

train_00000.png  train_00632.png  train_01264.png  train_01896.png
train_00001.png  train_00633.png  train_01265.png  train_01897.png
train_00002.png  train_00634.png  train_01266.png  train_01898.png
train_00003.png  train_00635.png  train_01267.png  train_01899.png
train_00004.png  train_00636.png  train_01268.png  train_01900.png
train_00005.png  train_00637.png  train_01269.png  train_01901.png
train_00006.png  train_00638.png  train_01270.png  train_01902.png
train_00007.png  train_00639.png  train_01271.png  train_01903.png
train_00008.png  train_00640.png  train_01272.png  train_01904.png
train_00009.png  train_00641.png  train_01273.png  train_01905.png
train_00010.png  train_00642.png  train_01274.png  train_01906.png
train_00011.png  train_00643.png  train_01275.png  train_01907.png
train_00012.png  train_00644.png  train_01276.png  train_01908.png
train_00013.png  train_00645.png  train_01277.png  train_01909.png
train_00014.png  train_00646.png  train_01278.png  train_01910

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.preprocessing import image

from keras.layers import Dense, GlobalAveragePooling2D


import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Reshape, Dropout
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback

In [0]:
src = 'train_images/'
src2=''

In [6]:
train_img = os.listdir('train_images')
print ('iamge lies size', len(train_img))
img_list=[]
for i,img in enumerate(train_img):
    x = cv2.imread(src+img)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    #x = x/127.5 -1.
    img_list.append(x)
    
print ('done')

iamge lies size 2528
done


In [7]:
train_data = np.array(img_list)
print (train_data.shape)
#train_data =  train_data[:,:,:,np.newaxis]
print (train_data.shape)

(2528, 512, 512, 3)
(2528, 512, 512, 3)


In [8]:
'''
with open(src+'train_data.pickle', 'wb') as handle:
    pickle.dump(train_data, handle)
'''

"\nwith open(src+'train_data.pickle', 'wb') as handle:\n    pickle.dump(train_data, handle)\n"

In [9]:
print ('here')

here


In [10]:
df_y = pd.read_csv('train.csv')
print (df_y['Label'].value_counts())
train_y =  df_y['Label'].values

onehot_encoder = OneHotEncoder(sparse=False)
train_y = train_y.reshape(len(train_y), 1)
print (train_y.shape)
train_label = onehot_encoder.fit_transform(train_y)


print ('beofre', train_label.shape)
print (df_y['Label'][:10])
print (train_label[:10,:])

train_label = train_label[:,np.newaxis,np.newaxis,:]

print ('after', train_label.shape)


0    674
5    644
1    492
3    378
4    240
2    100
Name: Label, dtype: int64
(2528, 1)
beofre (2528, 6)
0    0
1    1
2    1
3    5
4    5
5    5
6    3
7    0
8    3
9    5
Name: Label, dtype: int64
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
after (2528, 1, 1, 6)


In [0]:
class CollectOutputAndTarget(Callback):
    def __init__(self):
        super(CollectOutputAndTarget, self).__init__()
        self.targets = []  # collect y_true batches
        self.outputs = []  # collect y_pred batches

        # the shape of these 2 variables will change according to batch shape
        # to handle the "last batch", specify `validate_shape=False`
        self.var_y_true = tf.Variable(0., validate_shape=False)
        self.var_y_pred = tf.Variable(0., validate_shape=False)

    def on_epoch_end(self, batch, logs=None):
        # evaluate the variables and save them into lists
        '''
        print ('\n 0', len(self.validation_data))
        print ('1',  self.validation_data[0].shape )
        print ('2',  self.validation_data[1].shape )
        print ( '3', self.validation_data[2].shape )
        
        print ('4', test.shape)
       
        print ('5', test2.shape)
        
        print ('6', test3.shape)
        '''
        test = self.model.predict(self.validation_data[0])
        test2 = np.squeeze(test)
        test3 = np.argmax(test2,axis=1)
        print ('y_pred = \n', test3[:30])
        test_4 = np.squeeze(self.validation_data[1])
        test_5 = np.argmax(test_4,axis=1)
        print ('y_true = \n', test_5[:30])
        
        #print ('6', np.argmax(np.squeeze(test),axis=1))
        #out_result = np.argmax(np.squeeze(self.model.predict(self.validation_data[0])),axis=1)
        #ground_truth = np.argmax(np.squeeze (self.validation_data[1]), axis=1)



In [0]:
inputs = Input(shape=(512,512,3))
inputs = Lambda(lambda x: x/275. - 1.0)(inputs)
# create the base pre-trained model
base_model = DenseNet121(weights='imagenet',input_tensor=inputs, input_shape=(512,512,3), include_top=False)
for layer in base_model.layers:
    layer.trainable = False

In [16]:
# add a global spatial average pooling layer
x = base_model.output
print ('shape of densenet output', x.get_shape())
x = MaxPooling2D(pool_size=(2,2))(x)

#x = GlobalAveragePooling2D()(x)
print ('shape of pooling output', x.get_shape())
# let's add a fully-connected layer
x = Conv2D(32,(5,5), activation='relu', padding='valid')(x)
x = Conv2D(128,(3,3), activation='relu', padding='valid')(x)
x = Conv2D(128,(2,2), activation='relu', padding='valid')(x)
print ('shape of extra conv output', x.get_shape())
#predictions = Conv2D(6,(1,1), activation='relu', padding='same')(x)
x = Dense(256, activation='relu')(x)
# add Dropout
x = Dropout(0.3)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(6, activation='softmax')(x)
predictions = Reshape(target_shape=(1,1,6))(predictions)

shape of densenet output (?, 16, 16, 1024)
shape of pooling output (?, 8, 8, 1024)
shape of extra conv output (?, 1, 1, 128)


In [22]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (

In [0]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='Nadam', loss='categorical_crossentropy',metrics=['accuracy'])

#output_layers = ['predictions']
#model.metrics_tensors += output_layers


In [19]:

# initialize the variables and the `tf.assign` ops
cbk = CollectOutputAndTarget()
'''
fetches = [tf.assign(cbk.var_y_true, model.targets[0], validate_shape=False),
           tf.assign(cbk.var_y_pred, model.outputs[0], validate_shape=False)]

model._function_kwargs = {'fetches': fetches}  # use `model._function_kwargs` if using `Model` instead of `Sequential`
'''

"\nfetches = [tf.assign(cbk.var_y_true, model.targets[0], validate_shape=False),\n           tf.assign(cbk.var_y_pred, model.outputs[0], validate_shape=False)]\n\nmodel._function_kwargs = {'fetches': fetches}  # use `model._function_kwargs` if using `Model` instead of `Sequential`\n"

In [0]:
src= 'drive/aoi_test/'

In [21]:
# train the model on the new data for a few epochs


if os.path.isfile(src+'densenet_30_test.h5'):

  model = load_model(src+'densenet_30_test.h5')
  print ('model laoded')
else:
  print ('model not exist')


KeyboardInterrupt: ignored

In [0]:
checkpoint = ModelCheckpoint(src+'densenet_test_3.h5')
#callback_list = [cbk, checkpoint]
callback_list = [checkpoint]
for i in range (100):


  
  train_x,val_x,train_y,val_y = train_test_split(train_data,train_label,test_size=0.2, shuffle=True)
  
  model.fit(x=train_x, y=train_y, batch_size=32, epochs=10, verbose=1, callbacks=callback_list, validation_split=0.2, shuffle=True)
  #print ('epoch', i, 'shape=', len(cbk.outputs))


  model.save(src+'densenet_30_test_3.h5')
  print ('epoch', i, 'model saved')







Train on 1617 samples, validate on 405 samples
Epoch 1/10
1472/1617 [==========================>...] - ETA: 6s - loss: 2.0244 - acc: 0.2582

In [0]:
def shuffle_list(s,l):
    final_list = []
    n_list = np.arange(l)
    np.random.shuffle(n_list)
    b_s = int(np.floor(l/s))
    #print (b_s)
    
    for i in range (b_s):
        #print (i)
        final_list.append(n_list[i*s:(i+1)*s])
    return final_list

In [0]:
train_x,val_x,train_y,val_y = train_test_split(train_data,train_label,test_size=0.2, shuffle=True)

l = len(train_x)
s= 32

for epoch in range (100):

  batch_list = shuffle_list(s,l)
  for batch in batch_list:
    
    X = train_x[batch]
    Y = train_y[batch]
    [loss, accuracy]= model.train_on_batch(X,Y)

    print('during training: loss is {} accuracy is {}'.format(loss, accuracy))
    print ('prediction')
    print (np.argmax(np.squeeze(model.predict(X)),axis=-1))
    print ('ground truth')
    print (np.argmax(np.squeeze(Y),axis=-1))
  model.save(src+'densenet_30_test_2.h5')
  print ('epoch', epoch, 'model saved')
